# Choose Model Llama 3.2 with Fine-Tuning and Lora Layers

In [1]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


# Question test

In [2]:
# Example prompt for the LLM
question_data = {
    'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
    'option 1': 'PBCH',
    'option 2': 'PCFICH',
    'option 3': 'PDSCH',
    'option 4': 'PHICH',
}

# Format question and options into the prompt
question = question_data['question']
options = [f"{key}: {value}" for key, value in question_data.items() if 'option' in key]

prompt = (
    f"Question: {question}\n"
    f"Options:\n" + "\n".join(options) + "\n"
    # "Respond with the correct option in the format 'Final Answer: option <X>'.\n"
)

# Model Agent

In [3]:
from langchain_ollama import OllamaLLM

# Initialize the Ollama model with LLaMA 3.1 8B
llm = OllamaLLM(model="llama3.2")

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    # model="llama-3.1-70b-versatile",
    model="llama3-70b-8192",
    # model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Venturus API Llama

In [5]:
from venturus_api import call_llama

call_llama(prompt)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from typing import Optional, List

# Custom LLM class to wrap your call_llama function
class CustomLlamaLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Call your function and return the result
        response = call_llama(prompt)
        return response
    
    @property
    def _llm_type(self) -> str:
        return "custom_llama"

# Initialize the custom LLM
llm = CustomLlamaLLM()

# RAG Function

In [ ]:
from rag_functions import load_faiss_index, search_faiss_index, search_RAG, load_chunks

/home/arimatea/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-12 08:59:11.974711: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 08:59:12.112986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 08:59:12.174779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already bee

In [ ]:
result = search_RAG(prompt)
print(result)

Information 1:
of OFDM symbols of the PUSCH, including all OFDM symbols used for DMRS;  
\- for any OFDM symbol that carries DMRS of the PUSCH,
$M_{\text{sc}}^{\text{UCI}}\left( l \right) = 0$;  
\- for any OFDM symbol that does not carry DMRS of the PUSCH,
$M_{\text{sc}}^{\text{UCI}}\left( l \right) = M_{\text{sc}}^{\text{PUSCH}} - \ M_{\text{sc}}^{PT - RS}\left( l \right)$;  
\- $\alpha$ is configured by higher layer parameter *scaling*;  
\- $l_{0}$ is the symbol index of the first OFDM symbol that does not
carry DMRS of the PUSCH, after the first DMRS symbol(s), in the PUSCH
transmission.  
For CG-UCI transmission on PUSCH with UL-SCH, and if
*numberOfSlotsTBoMS* is present in the resource allocation table and the
value of *numberOfSlotsTBoMS* in the row indicated by the Time domain
resource assignment field in DCI is larger than 1, the number of coded
modulation symbols per layer for CG-UCI transmission, denoted as
$Q_{CG - UCI}^{'}$, is determined as follows:  
$$Q_{CG - UCI}^{'}

# TeleQnA 100 questions

In [ ]:
import json

# Path to the TeleQnA processed question in JSON file
rel17_100_questions_path = r"../../Files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_100_questions_path, "r", encoding="utf-8") as file:
    rel17_100_questions = json.load(file)
print(len(rel17_100_questions))

100


In [ ]:
rel17_100_questions[18]

{'question': 'What aspects of the Disaster Roaming service are specified in TS 22.011 and TS 22.261? [3GPP Release 17]',
 'option 1': 'Disaster Condition configuration procedures.',
 'option 2': 'Disaster Roaming service registration procedures.',
 'option 3': 'System information extensions for MINT.',
 'option 4': 'Network selection procedures.',
 'option 5': 'Service requirements for the 5G system.',
 'answer': 'option 5: Service requirements for the 5G system.',
 'explanation': 'The service requirements for the 5G system are specified in TS 22.011 and TS 22.261.',
 'category': 'Standards overview'}

# Other Agent

In [ ]:
# #@title Define agent

# # Adapted from: How to Create your own LLM Agent from Scratch: A Step-by-Step Guide
# # Link: https://gathnex.medium.com/how-to-create-your-own-llm-agent-from-scratch-a-step-by-step-guide-14b763e5b3b8

In [ ]:
# import re

# # Função para gerar o prompt com a questão de entrada e o rascunho inicial do agente
# def generate_prompt(input_question, agent_scratchpad, temporary_message=""):
#     template = f"""
# Answer the following questions as best you can. You have access to the following tools:

# retrieval

# Follow this process until you have an answer:

# 1. Question: the input question you must answer. Decompose in more than one question if needed.
# 2. Thought: you should always think about what to do. Answer the question or do the search.
# If you do the search provide:
# 3. Action: retrieval
# 4. Action Input: the input to the action
# After give the search information wait the observation.
# 5. Observation: the result of the action

# When you have an answer, give the Final Answer:
# Final Answer: option <X>

# You only give a Final Answer if you are certain!
# You can choose only One Option!

# Begin!

# {input_question}
# Thought: {agent_scratchpad}
# {temporary_message}
# """
#     # return template.format(input_question=input_question, agent_scratchpad=agent_scratchpad, temporary_message=temporary_message)
#     return template

# # Funções para extrair a ação e o input do agente
# def extract_action(text):
#     action_pattern = r'Action:\s*(\S+)'  
#     action = re.search(action_pattern, text)
#     return action.group(1).strip().lower() if action else None

# def extract_input(text):
#     input_pattern = r'Action Input:\s*(.*?)(?=\n|$)'  
#     action_input = re.search(input_pattern, text, re.DOTALL)
#     return action_input.group(1).strip() if action_input else None

# def extract_response(text):
#     response_pattern = r'Final Answer:\s*option\s*(\d+)'  
#     response = re.search(response_pattern, text)
#     return f"option {response.group(1)}" if response else None

# # Função de feedback para Action Input inválido
# def feedback_missing_action_input():
#     return (
#         "Feedback: The Action Input cannot be None. Please provide a valid input for the search "
#         "that will help answer the question.\n"
#         "Expected format: Action: retrieval, Action Input: <your query>"
#     )

# # Função de feedback para ação inválida
# def feedback_invalid_action():
#     return (
#         "Feedback: The action provided is invalid. Please provide a valid action.\n"
#         "Expected Action: retrieval. Please also provide an appropriate Action Input."
#     )

# # Função principal para executar o agente com limite de passos e depuração
# def Stream_agent(query, max_steps=5, debug=False):
#     agent_scratchpad = "Initial thought process begins here."

#     steps = 0
#     while True:
        
#         messages = [
#         {"role": "system", "content": generate_prompt(query, agent_scratchpad)},
#         # {"role": "user", "content": query},
#         ]
        
#         prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        
#         if debug:
#             print("\n" + "=" * 50)
#             print(f"[DEBUG] Prompt:\n{prompt}")
#             print("\n" + "=" * 50)
        
#         response_message = llm.invoke(prompt)  # Invoke LLM with prompt (substitute with your LLM function)
        
#         if debug:
#             print(f"\n>> [DEBUG] Agent response: {response_message}")

#         while True:
#             action = extract_action(response_message)
#             action_input = extract_input(response_message)
            
#             if debug:
#                 print(f"[DEBUG] Action: {action}")
#                 print(f"[DEBUG] Action Input: {action_input}")

#             if action == "retrieval" and action_input:
#                 observation = search_RAG(action_input)  # Replace with actual search function
#                 # if debug:
#                 #     print(f"[DEBUG] Observation: {observation}")

#                 if observation:
#                     agent_scratchpad += f"\nObservation:\n{observation}"
#                     temporary_message = (
#                         # f"Observation:\n{observation}\n"
#                         "\nThink step by step and analyze the observation.\n"
#                         "If you are certain of the answer, please provide a response in the following format: Final Answer: option <x>.\n"
#                         "If you are not certain and need another search, say: I will do another search."
#                     )
#                     prompt = generate_prompt(query, agent_scratchpad, temporary_message)
                    
#                     if debug:
#                         print("\n" + "=" * 50)
#                         print(f"[DEBUG] Prompt:\n{prompt}")
#                         print("\n" + "=" * 50)
                    
#                     response_message = llm.invoke(prompt)

#                     response = extract_response(response_message)
                    
#                     if response:
#                         if debug:
#                             print(f"[DEBUG] Final Answer: {response}")
#                         return response
#                     else:
#                         if debug:
#                             print(f"[DEBUG] Agent response: {response_message}")
#                         agent_scratchpad += f"\nDo another search\nProvide the informations:\nAction: retrieval\nAction Input: <your query>"
#                         # messages.append({"role": "user", "content": "Please try again with another search."})
#                 break 

#             else:
#                 feedback = feedback_invalid_action() if action != "retrieval" else feedback_missing_action_input()
#                 # messages.append({"role": "user", "content": feedback})
#                 if debug:
#                     print(f"[DEBUG] Providing Feedback: {feedback}")
#                 break  

#         steps += 1
#         if steps >= max_steps:
#             if debug:
#                 print("Maximum steps reached. Stopping.")
#             return None

In [ ]:
import re

# Função para gerar o prompt com a questão de entrada e o rascunho inicial do agente
def generate_prompt(input_question, agent_scratchpad, temporary_message=""):
    template = f"""
Answer the following questions as best you can. You have access to the following tools:

retrieval

Follow this process until you have an answer:

1. Question: the input question you must answer. Decompose in more than one question if needed.
2. Thought: you should always think about what to do. Answer the question or do the search.
If you do the search provide:
3. Action: retrieval
4. Action Input: the input to the action
After give the search information wait the observation.
5. Observation: the result of the action

When you have an answer, give the Final Answer:
Final Answer: option <X>

You only give a Final Answer if you are certain!
You can choose only One Option!

Begin!

{input_question}
Thought: {agent_scratchpad}
{temporary_message}
"""
    # return template.format(input_question=input_question, agent_scratchpad=agent_scratchpad, temporary_message=temporary_message)
    return template

# Funções para extrair a ação e o input do agente
def extract_action(text):
    action_pattern = r'Action:\s*(\S+)'  
    action = re.search(action_pattern, text)
    return action.group(1).strip().lower() if action else None

def extract_input(text):
    input_pattern = r'Action Input:\s*(.*?)(?=\n|$)'  
    action_input = re.search(input_pattern, text, re.DOTALL)
    return action_input.group(1).strip() if action_input else None

def extract_response(text):
    response_pattern = r'Final Answer:\s*option\s*(\d+)'  
    response = re.search(response_pattern, text)
    return f"option {response.group(1)}" if response else None

# Função de feedback para Action Input inválido
def feedback_missing_action_input():
    return (
        "Feedback: The Action Input cannot be None. Please provide a valid input for the search "
        "that will help answer the question.\n"
        "Expected format: Action: retrieval, Action Input: <your query>"
    )

# Função de feedback para ação inválida
def feedback_invalid_action():
    return (
        "Feedback: The action provided is invalid. Please provide a valid action.\n"
        "Expected Action: retrieval. Please also provide an appropriate Action Input."
    )

# Função principal para executar o agente com limite de passos e depuração
def Stream_agent(query, max_steps=5, debug=False):
    agent_scratchpad = "Initial thought process begins here."

    steps = 0
    observation_idx = 0
    agent_observation = ""
    while True:
        
        messages = [
        {"role": "system", "content": generate_prompt(query, agent_scratchpad)},
        # {"role": "user", "content": query},
        ]
        
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        
        if debug:
            print("\n" + "=" * 50)
            print(f"[DEBUG] Prompt:\n{prompt}")
            print("\n" + "=" * 50)
        
        response_message = llm.invoke(prompt)  # Invoke LLM with prompt (substitute with your LLM function)
        
        if debug:
            print(f"\n>> [DEBUG] Agent response: {response_message}")
            
        observation_idx += 1
        
        agent_scratchpad += f"Thought: {response_message}"

        while True:
            action = extract_action(response_message)
            action_input = extract_input(response_message)
            
            if debug:
                print(f"[DEBUG] Action: {action}")
                print(f"[DEBUG] Action Input: {action_input}")

            if action == "retrieval" and action_input:
                observation = search_RAG(action_input)  # Replace with actual search function
                # if debug:
                #     print(f"[DEBUG] Observation: {observation}")

                if observation:
                    agent_observation += f"\nObservation {observation_idx}:\n{observation}\n"
                    temporary_message = (
                        f"Observation:\n{agent_observation}\n"
                        f"{query}"
                        "\nThink step by step and analyze the observation.\n"
                        "If you are certain of the answer, please provide a response in the following format: Final Answer: option <x>.\n"
                        "If you are not certain and need another search, say: I will do another search."
                    )
                    # prompt = generate_prompt(query, agent_scratchpad, temporary_message)
                    prompt = temporary_message
                    
                    if debug:
                        print("\n" + "=" * 50)
                        print(f"[DEBUG] Prompt:\n{prompt}")
                        print("\n" + "=" * 50)
                    
                    response_message = llm.invoke(prompt)

                    response = extract_response(response_message)
                    
                    if response:
                        if debug:
                            print(f"[DEBUG] Final Answer: {response}")
                        return response
                    else:
                        if debug:
                            print(f"[DEBUG] Agent response: {response_message}")
                        agent_scratchpad += f"\n{response_message}\nDo another search\nProvide the informations:\nAction: retrieval\nAction Input: <your query>"
                        # messages.append({"role": "user", "content": "Please try again with another search."})
                break 

            else:
                feedback = feedback_invalid_action() if action != "retrieval" else feedback_missing_action_input()
                # messages.append({"role": "user", "content": feedback})
                if debug:
                    print(f"[DEBUG] Providing Feedback: {feedback}")
                break  

        steps += 1
        if steps >= max_steps:
            if debug:
                print("Maximum steps reached. Stopping.")
            return None

In [ ]:
def ask_agent(question_data):
    """
    Function to generate an answer using the model based on the given question and options, 
    including relevant information from a RAG search and prompting a chain of thought.
    
    Parameters:
    - llm: The language model loaded for inference.
    - question_data: Dictionary containing the question and options.

    Returns:
    - String: Model's generated response.
    """

    # Extract question and options
    question = question_data['question']
    options = [f"{key}: {value}" for key, value in question_data.items() if 'option' in key]
    
    prompt = (
        f"Question: {question}\n"
        f"Options:\n" + "\n".join(options) + "\n"
        # "Give only one option and do a search if you need"
    )

    response = Stream_agent(query=prompt, max_steps=10, debug=True)
    
    return response


In [ ]:
rel17_100_questions[18]

{'question': 'What aspects of the Disaster Roaming service are specified in TS 22.011 and TS 22.261? [3GPP Release 17]',
 'option 1': 'Disaster Condition configuration procedures.',
 'option 2': 'Disaster Roaming service registration procedures.',
 'option 3': 'System information extensions for MINT.',
 'option 4': 'Network selection procedures.',
 'option 5': 'Service requirements for the 5G system.',
 'answer': 'option 5: Service requirements for the 5G system.',
 'explanation': 'The service requirements for the 5G system are specified in TS 22.011 and TS 22.261.',
 'category': 'Standards overview'}

In [ ]:
question_data = rel17_100_questions[18]

agent_response = ask_agent(question_data)


[DEBUG] Prompt:
system: 
Answer the following questions as best you can. You have access to the following tools:

retrieval

Follow this process until you have an answer:

1. Question: the input question you must answer. Decompose in more than one question if needed.
2. Thought: you should always think about what to do. Answer the question or do the search.
If you do the search provide:
3. Action: retrieval
4. Action Input: the input to the action
After give the search information wait the observation.
5. Observation: the result of the action

When you have an answer, give the Final Answer:
Final Answer: option <X>

You only give a Final Answer if you are certain!
You can choose only One Option!

Begin!

Question: What aspects of the Disaster Roaming service are specified in TS 22.011 and TS 22.261? [3GPP Release 17]
Options:
option 1: Disaster Condition configuration procedures.
option 2: Disaster Roaming service registration procedures.
option 3: System information extensions for MI

In [ ]:
agent_response

'option 1'

# Ask 50 questions to Stream Agent

In [ ]:
def evaluate_questions(questions):
    """
    Process all questions and return the model responses.
    
    Parameters:
    - questions: List of dictionaries containing question data, where each dictionary has:
        - 'question': A string representing the question to be asked to the model.
        - 'answer': A string representing the correct answer format (e.g., 'option 2: PCFICH').
        - 'response': A string that will contain the model's generated response to the question.
    
    Returns:
    - List: A list of dictionaries where each dictionary contains:
        - 'question': The question as a string.
        - 'answer': The correct answer as a string.
        - 'response': The model's generated response for that question.
    """
    
    responses = []
    total_questions = len(questions)
    
    for idx, question_data in enumerate(questions):
        response = ask_agent(question_data)
        responses.append({
            "question": question_data['question'],
            "answer": question_data['answer'],
            "response": response
        })
        
        # Print progress
        print(f"Responded {idx + 1} of {total_questions} questions...")

    return responses

In [ ]:
# Process all questions and get responses
responses_agent = evaluate_questions(rel17_100_questions[:100])


[DEBUG] Prompt:
system: 
Answer the following questions as best you can. You have access to the following tools:

retrieval

Follow this process until you have an answer:

1. Question: the input question you must answer. Decompose in more than one question if needed.
2. Thought: you should always think about what to do. Answer the question or do the search.
If you do the search provide:
3. Action: retrieval
4. Action Input: the input to the action
After give the search information wait the observation.
5. Observation: the result of the action

When you have an answer, give the Final Answer:
Final Answer: option <X>

You only give a Final Answer if you are certain!
You can choose only One Option!

Begin!

Question: Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]
Options:
option 1: eNB Configuration Transfer
option 2: Downlink RAN Configuration Transfer
option 3: Uplink RAN Configuration Transfer
option 4: MME Configuration Transfer

Thought: Initial thou

In [ ]:
responses_agent[0]

{'question': 'Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]',
 'answer': 'option 3: Uplink RAN Configuration Transfer',
 'response': 'option 4'}

# Save Agent responses

In [ ]:
def save_responses_to_json(responses, filename):
    """
    Save the model responses to a JSON file.
    
    Parameters:
    - responses: List of responses to save.
    - filename: Name of the JSON file.
    """
    
    with open(filename, "w") as json_file:
        json.dump(responses, json_file, indent=4)

In [ ]:
# save_responses_to_json(responses_agent,"../../Models_responses/Accuracy/agent_llama_3_2_responses.json")

# Evaluate responses from Agent

In [ ]:
import json

# Load responses from the JSON file
with open("../../Models_responses/Accuracy/agent_llama_3_2_responses.json", "r") as file:
    responses_agent = json.load(file)

print("Responses loaded")
print(f"{len(responses_agent)} responses")


Responses loaded
100 responses


In [ ]:
import re

def extract_option(answer):
    """
    Extract the option part from the answer string, removing all punctuation and converting to lowercase.
    
    Parameters:
    - answer: A string containing the answer in the format 'option X: ...'.

    Returns:
    - String: Extracted option (e.g., 'option 2'), or None if no match is found.
    """
    if answer is not None:
        # Remove all punctuation and convert to lowercase
        cleaned_answer = re.sub(r'[^\w\s]', '', answer.lower()) 
        # Search for the option in the format "option X"
        match = re.search(r'option \d+', cleaned_answer)
        return match.group(0).strip() if match else None
    else:
        return None

In [ ]:
def evaluate_model_response(model_response, question_data):
    """
    Compare the model's response with the correct answer from the question data.
    
    Parameters:
    - model_response: The response string generated by the model.
    - question_data: Dictionary containing the question, options, and the correct answer.

    Returns:
    - 1 if the response is correct, otherwise the extracted model option.
    """
    correct_option = extract_option(question_data['answer'])  # Extract correct option
    model_option = extract_option(model_response)  # Extract model's option

    return 1 if model_option == correct_option else model_option  # Return 1 if correct, else model's option

In [ ]:
def evaluate_accuracy(responses):
    """
    Evaluate the model's responses and calculate accuracy.
    """
    correct_count = 0  # Track the number of correct responses
    none_count = 0  # Track the number of 'None' responses

    for index, question_data in enumerate(responses):
        evaluation_result = evaluate_model_response(question_data['response'], question_data)
        options = [f"{key}: {value}" for key, value in rel17_100_questions[index].items() if 'option' in key]

        if evaluation_result == 1:
            correct_count += 1  # Increment for correct response
        elif evaluation_result is None:
            # Print only responses that are None
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Correct response: {question_data['answer']}")
            print(f"Full model response:\n{question_data['response']}")
            print("----------------------------------------------------------------------------------------")
            none_count += 1  # Increment for None response
        else:
            print("\nWrong Answer")
            print(f"Question {index + 1}: {question_data['question']}")
            print(f"Options:\n" + "\n".join(options) + "\n")
            print(f"Correct response: {question_data['answer']}")
            print(f"Model response: {evaluation_result}")
            print("----------------------------------------------------------------------------------------")

    # Calculate and print accuracy
    accuracy = correct_count / len(responses) * 100
    print(f"\nAccuracy: {accuracy:.2f}%")
    print(f"Total 'None' responses: {none_count}")
    print(f"'None' responses means that the model did not give an option")


In [ ]:
evaluate_accuracy(responses_agent)


Wrong Answer
Question 1: Which NGAP procedure is used for inter-system load balancing? [3GPP Release 17]
Options:
option 1: eNB Configuration Transfer
option 2: Downlink RAN Configuration Transfer
option 3: Uplink RAN Configuration Transfer
option 4: MME Configuration Transfer

Correct response: option 3: Uplink RAN Configuration Transfer
Model response: option 4
----------------------------------------------------------------------------------------

Wrong Answer
Question 3: What does the Load-Balancing steering mode do? [3GPP Release 17]
Options:
option 1: Splits the traffic of a data flow across 3GPP and non-3GPP accesses
option 2: Balances the load on the core network nodes
option 3: Restricts the UE to a single access type
option 4: Improves network performance measurements

Correct response: option 1: Splits the traffic of a data flow across 3GPP and non-3GPP accesses
Model response: option 4
---------------------------------------------------------------------------------------